In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import os

In [2]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

Data Preprocessing

In [3]:
df_train = pd.read_csv("C:/Users/absub/Downloads/sign_mnist_train/sign_mnist_train.csv")
df_test = pd.read_csv("C:/Users/absub/Downloads/sign_mnist_test/sign_mnist_test.csv")

In [4]:
df_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [5]:
xTrain = df_train.drop('label', axis=1).values.reshape(-1, 28, 28, 1)
xTest = df_test.drop('label', axis=1).values.reshape(-1, 28, 28, 1)
xTrain = xTrain / 255.0
xTest = xTest / 255.0

yTrain = pd.get_dummies(df_train['label']).values
yTest = pd.get_dummies(df_test['label']).values

print(xTrain.shape)
print(yTrain.shape)
print(xTest.shape)
print(yTest.shape)

(27455, 28, 28, 1)
(27455, 24)
(7172, 28, 28, 1)
(7172, 24)


In [6]:
dataAugment = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
)

In [7]:
xTrain, xVal, yTrain, yVal = train_test_split(xTrain,yTrain,test_size=0.2,random_state=42)

In [8]:
print(xTrain.shape)
print(yTrain.shape)
print(xVal.shape)
print(yVal.shape)

(21964, 28, 28, 1)
(21964, 24)
(5491, 28, 28, 1)
(5491, 24)


Creating the CNN

In [9]:
# Define the model
model = tf.keras.models.Sequential([
    # Input Layer
    tf.keras.layers.Input(shape=(28, 28, 1)),

    # Convolutional Block 1
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),

     # Convolutional Block 2
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(strides=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    # Classification Block
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),

     tf.keras.layers.Dense(24, activation='softmax')
])


# Compile Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 28, 28, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 28, 28, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 28, 28, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 28, 28, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 14, 14, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 14, 14, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 14, 14, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 14, 14, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 3136)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,606,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 24)                  │          12,312 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,686,264 (6.43 MB)

 Trainable params: 1,684,856 (6.43 MB)

 Non-trainable params: 1,408 (5.50 KB)

Training the model on the Training Data

In [10]:
train_generator = dataAugment.flow(xTrain, yTrain, batch_size=32)
validation_generator = dataAugment.flow(xVal, yVal, batch_size=32)


early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/50


C:\Users\absub\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


687/687 ━━━━━━━━━━━━━━━━━━━━ 122s 150ms/step - acc: 0.6161 - loss: 1.3225 - val_acc: 0.9153 - val_loss: 0.2577 - learning_rate: 0.0010
Epoch 2/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 102s 149ms/step - acc: 0.9465 - loss: 0.1666 - val_acc: 0.9792 - val_loss: 0.0656 - learning_rate: 0.0010
Epoch 3/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 107s 155ms/step - acc: 0.9731 - loss: 0.0810 - val_acc: 0.9015 - val_loss: 0.2995 - learning_rate: 0.0010
Epoch 4/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 84s 123ms/step - acc: 0.9763 - loss: 0.0727 - val_acc: 0.9942 - val_loss: 0.0242 - learning_rate: 0.0010
Epoch 5/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 84s 122ms/step - acc: 0.9858 - loss: 0.0484 - val_acc: 0.9785 - val_loss: 0.0674 - learning_rate: 0.0010
Epoch 6/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 85s 123ms/step - acc: 0.9861 - loss: 0.0447 - val_acc: 0.9667 - val_loss: 0.0984 - learning_rate: 0.0010
Epoch 7/50
687/687 ━━━━━━━━━━━━━━━━━━━━ 84s 122ms/step - acc: 0.9886 - loss: 0.0349 - val_acc: 0.9925 - val_loss: 0.0218 - learning_rate: 

Tesing the model on the Testing Data

In [11]:
model.evaluate(xTest, yTest)

225/225 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - acc: 1.0000 - loss: 8.1122e-05


[0.000224697170779109, 1.0]

In [12]:
from sklearn.metrics import accuracy_score

y_preds = model.predict(xTest)
max_indices = np.argmax(y_preds, axis=1)
y_preds_bool = np.zeros_like(y_preds, dtype=bool)
y_preds_bool[np.arange(y_preds.shape[0]), max_indices] = True

accuracy = accuracy_score(yTest, y_preds_bool)
print(f"Model Accuracy: {accuracy*100:.20f}%")

225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step
Model Accuracy: 100.00000000000000000000%


In [13]:
# Saving the full model
model.save("model_sign_language.h5")

In [14]:
# Save the full model in the Keras format
model.save("model_sign_language.keras")